# Выбор локации для скважины

## Введение

Данное исследование проводится с целью определение региона, добыча нефти в котором потенциально принесет наибольшую прибыль компании ГлавРосГосНефть. В ходе работы над исследованием с помощью модели линейной регрессии будет предсказан объем запасов в скважинах для каждого региона, после чего будет рассчитан объём сырья, достаточный для безубыточной разработки новой скважины. Далее будут выбраны наиболее перспективные с точки зрения объема сырья скважины, для этого объема будет посчитана прибыль. Наконец, будет произведена оценка оценка риска и прибыли для каждого региона.

## 1. Загрузка и предобработка данных

Для начала подключим необходимые в дальнейшей работе библиотеки:

In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from scipy import stats as st

Теперь загрузим данные, необходимые для исследования:

In [48]:
try:
    geo_data_0 = pd.read_csv('geo_data_0.csv')
except:
    geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
    
try:
    geo_data_1 = pd.read_csv('geo_data_1.csv')
except:
    geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
    
try:
    geo_data_2 = pd.read_csv('geo_data_2.csv')
except:
    geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

Проведем обзор первых 5 строчек каждого датафрейма:

In [49]:
display('geo_data_0:', geo_data_0.head(), 'geo_data_1:', geo_data_1.head(), 'geo_data_2:', geo_data_2.head())

'geo_data_0:'

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


'geo_data_1:'

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


'geo_data_2:'

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Удалим из каждого набора столбец 'id', так как он не несет в себе информации, необходимой для дальнейшего обучения модели линейной регрессии:

In [50]:
geo_data_0 = geo_data_0.drop('id', axis=1)
geo_data_1 = geo_data_1.drop('id', axis=1)
geo_data_2 = geo_data_2.drop('id', axis=1)

Проверим данные на наличие пропусков, для чего посмотрим на общую информацию каждого из трех датасетов:

In [51]:
geo_data_0.info(), geo_data_1.info(), geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1 

(None, None, None)

Наконец, проверим данные на наличие дубликатов:

In [52]:
print('Дубликатов в geo_data_0 -', geo_data_0.duplicated().sum(), \
      '\nДубликатов в geo_data_1 -', geo_data_1.duplicated().sum(), \
      '\nДубликатов в geo_data_2 -', geo_data_2.duplicated().sum())

Дубликатов в geo_data_0 - 0 
Дубликатов в geo_data_1 - 0 
Дубликатов в geo_data_2 - 0


Итак, мы удалили из данных неинформативный признак 'id', убедились в том, что они не содержат в себе пропусков и дубликатов - данные подготовлены для дальнейшей работы.

## 2. Обучение и проверка моделей

Напишем функцию, которая произведет разбивку данных на тренировочную и валидационную выборки, обучит модель линейной регрессии и сделает предсказание целевого признака на валидационном наборе. Возвращать функция будет предсказания вместе с правильными ответами. Для удобства дальнейшей работы переведем предсказания в Series и обновим индексы для целевого признака:

In [53]:
def predictor (df):
    features = df.drop('product', axis=1)
    target = df['product']

    features_train, features_valid, target_train, target_valid = train_test_split(features.copy(), target.copy(), 
                                                                              test_size=0.25, random_state=12345)
    model = LinearRegression()
    model.fit(features_train, target_train)
    
    predicted_valid = model.predict(features_valid)
    
    return pd.Series(predicted_valid), target_valid.reset_index(drop=True)

Для каждого региона **сохраним** предсказания и правильные ответы на валидационной выборке и выведем средний запас **предсказанного** сырья и RMSE модели:

In [54]:
predicted_valid_0, target_valid_0 = predictor(geo_data_0)
print(f'Средний запас предсказанного сырья в первом регионе равен {predicted_valid_0.mean()}\
, а RMSE модели - {mean_squared_error(target_valid_0, predicted_valid_0) ** 0.5}')

Средний запас предсказанного сырья в первом регионе равен 92.59256778438035, а RMSE модели - 37.5794217150813


In [55]:
predicted_valid_1, target_valid_1 = predictor(geo_data_1)
print(f'Средний запас предсказанного сырья во втором регионе равен {predicted_valid_1.mean()}\
, а RMSE модели - {mean_squared_error(target_valid_1, predicted_valid_1) ** 0.5}')

Средний запас предсказанного сырья во втором регионе равен 68.728546895446, а RMSE модели - 0.893099286775617


In [56]:
predicted_valid_2, target_valid_2 = predictor(geo_data_2)
print(f'Средний запас предсказанного сырья в третьем регионе равен {predicted_valid_2.mean()}\
, а RMSE модели - {mean_squared_error(target_valid_2, predicted_valid_2) ** 0.5}')

Средний запас предсказанного сырья в третьем регионе равен 94.96504596800489, а RMSE модели - 40.02970873393434


Как видим, наименьший корень из среднеквадратичной ошибки получился у модели линейной регрессии, обученной на данных для второго региона, значение его составило 0.89. Средний предсказанный запас в этом регионе составил 68.73 тысяч баррелей на скважину. Низкое значение ошибки говорит о том, что среднее значение, вычисленное на предсказаниях модели не будет сильно отличаться от того же значения, посчитанного на реальных данных. То есть, предсказаниям модели для этого региона мы можем доверять больше, чем предсказаниям для первого и третьего регионов. 

## 3. Подготовка к расчету прибыли

Проведем рассчет, чтобы найти достаточный объём сырья для безубыточной разработки новой скважины и сравним объём сырья со средним запасом в каждом регионе.  

Напомним себе условия бизнеса. Для разработки в регионе выбирают 200 лучших (по предсказанию моделей машинного обучения) скважин, бюджет на их разработку составляет 10 млрд рублей. Один баррель сырья приносит 450 рублей дохода, то есть, доход с каждой единицы продукта составляет 450 тыс. рублей (объём запасов в столбце 'product' указан в тысячах баррелей). Сохраним эти данные в отдельных переменных:

In [57]:
wells_to_develop = 200 # Скважин для разработки
development_budget = 10000000000 # Бюджет на разработку
income_per_barrel = 450 # Доход на баррель
barrels_in_product_unit = 1000 # Баррелей на единицу продукта

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Для этого разделим общий бюджет на количество разрабатываемых скважин и на доход с одной единицы продукта (1000 баррелей). Таким образом узнаем, каков должен быть объем запасов в скважинах, чтобы наши расходы их разработку "вышли в ноль".

In [58]:
enough_volume = development_budget / (wells_to_develop * income_per_barrel * barrels_in_product_unit)
enough_volume

111.11111111111111

Итак, по нашим расчетам, для безубыточной разработки объем сырья в скважине должен составлять 111 тысяч баррелей, что больше предсказанного моделью линейной регрессии **среднего** объема в каждом регионе. 

На данном этапе, сохранив нужные для расчётов значения в соответствующих переменных, мы рассчитали минимальный объем сырья в скважине, позволяющий вести неубыточную разработку. В итоге вышло, что искомый объем составляет 111 тысяч баррелей, данная величина превосходит средние объемы для всех трех регионов, ближе всего к ней приближается первый регион, средний объем предсказанного на предыдущем этапе исследования сырья в котором составил 92 592 барреля.

## 4. Написание функции для расчета прибыли

Для более удобного рассчета прибыли напишем функцию, которая на вход будет принимать предсказанный объем сырья в каждой из скважин и фактическое количество сырья в этих скважинах. Отсортируем предсказания в порядке убывания значений запасов продукта, выберем 200 наибольших значений, которые просуммируем и умножим на доход, получаемый с единицы продукта - так получим выручку. Чтобы вычислить прибыль, вычтем из выручки сумму, потраченную на разработку скважин. Необходимые значения получим из заранее созданным переменных:

In [59]:
def profit_calculator (predictions, target):
    
    predictions_sorted = predictions.sort_values(ascending=False)
    top_wells = target[predictions_sorted.index][:wells_to_develop]
    top_wells_total_product = top_wells.sum()
    
    return top_wells_total_product * income_per_barrel * barrels_in_product_unit - development_budget

## 5. Подсчет рисков и прибыли

Напишем функцию, которая с помощью техники Bootstrap позволит нам определить среднюю прибыль, 95%-й доверительный интервал и риск убытков. Напишем функцию, принимающую на вход предсказанные объемы сырья в скважине и "правильные ответы". Далее с помощью цикла на 1000 итераций применим технику Bootstrap, посчитав для подвыборок размера 500 исходных датафреймов прибыль с помощью функции, написанной на прошлом шаге. После этого вычислим среднее значение всех прибылей, необходимый доверительный интервал и риски:

In [60]:
def bootstrap (predictions, target): 

    state = np.random.RandomState(12345)

    profits = []

    for i in range(1000):
    
        subsample_target = target.sample(n=500, replace=True, random_state=state)
        subsample_predicted = predictions[subsample_target.index]
    
        profits.append(profit_calculator(subsample_predicted, subsample_target))
    
    profits = pd.Series(profits)
    
    mean_profit = profits.mean()
    confidence_interval_95 = st.t.interval(confidence=0.95, df=len(profits)-1, loc=profits.mean(), scale=profits.std())
    loss_risk = profits[profits < 0].count() / len(profits) * 100
    
    return mean_profit, confidence_interval_95, loss_risk

Теперь выведем искомые значения, для удобства воспользовавшись циклом:

In [61]:
pairs = [[predicted_valid_0, target_valid_0], [predicted_valid_1, target_valid_1], [predicted_valid_2, target_valid_2]]
for i in range(3):
    mean, confidence_interval, risk = bootstrap(pairs[i][0], pairs[i][1])
    print(f'Средняя прибыль для {i + 1} региона - {mean}')
    print(f'95%-й доверительный интервал - {confidence_interval}')
    print(f'Риск получить убытки - {risk}%')
    print()

Средняя прибыль для 1 региона - 425938526.91059244
95%-й доверительный интервал - (-118173081.58673334, 970050135.4079182)
Риск получить убытки - 6.0%

Средняя прибыль для 2 региона - 515222773.4432898
95%-й доверительный интервал - (85111988.47686392, 945333558.4097157)
Риск получить убытки - 1.0%

Средняя прибыль для 3 региона - 435008362.7827556
95%-й доверительный интервал - (-120123495.57730567, 990140221.1428169)
Риск получить убытки - 6.4%



Как со всей очевидностью можно судить по выводу предыдущей ячейки, **лучший кандидат для разработки скважин - второй регион**. Он лидирует по всем трем позициям - обладает наивысшей средней прибылью (515 миллионов), его доверительный интервал содержит в себе наибольшие значения, причем, положительные (с вероятностью в 95% среднее значение прибыли от добычи в этом регионе составит от 85 до 945 миллионов), разработка скважин в нем несет в себе меньше всего рисков (1%, что в 6 раз меньше, чем в двух остальных регионах).